## Formas de especificar um modelo

Neste exemplo, é mostrado como descrever e renderizar um objeto utilizando as primitivas de triângulo, linhas e pontos. Além disso, é mostrado também como utilizar a função 'glViewport()' para renderizar em diferentes porções da tela.

Há várias formas de descrever a malha triangular de um modelo. Uma dessas formas é definir um vetor contendo todas as coordenadas dos vértices do objeto em que três coordenadas consecutivas definem um triângulo (vetor com nome 'vertex_position' deste exemplo), como temos feito até agora.  Uma desvantagem dessa abordagem é que coordenadas comuns em mais de um triângulo aparecem repetidas no vetor de dados. Para evitar essa redundância e economizar memória, a malha triangular pode ser descrita utilizando dois vetores. Um desses vetores armazena, sem redundância, todas as coordenadas dos vértices do objeto (vetor com nome 'vertex_position_02' deste exemplo). E outro vetor armazena os índices dessas coordenadas em que três índices consecutivos definem um triângulo (vetor com nome 'vertex_indices' deste exemplo). Este notebook mostra estas duas abordagens. Perceba que o recurso de culling está ativado neste exemplo e que todos os vértices foram especificados no sentido anti-horário. Caso contrário, os triângulos não seriam renderizados.

Para desenhar linhas, os vértices do vetor 'vertex_position_02' foram renderizados utilizando as primitivas LINES e LINE_LOOPS como exemplo. A primeira renderiza uma nova linha a cada dois vértices. No caso do vetor 'vertex_position_02' que possui quatro vértices, duas linhas são renderizadas. A segunda primitiva renderiza uma linha que conecta um vértice com o vértice anterior e conecta o último vértice com o primeiro. No caso do vetor 'vertex_position_02', quatro linhas são renderizadas, formando um quadrado não preenchido. A abordagem de descrever um objeto utilizando um vetor de índices (apresentada na renderização de triângulos) para renderizar somente linhas pode ser usada em ambas as primitivas de linhas. No entanto, há economia de memória apenas com a primitiva LINES.

Por fim, para desenhar pontos, os vértices do vetor 'vertex_position_02' também foram utilizados, mas agora utilizando a primitiva POINTS, que renderiza um ponto na tela para cada vértice. Com esta primitiva, a abordagem de descrever um objeto utilizando um vetor de índices para renderizar somente pontos não apresenta benefícios.

In [3]:
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v2 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v2 import ModelRenderer

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        # posição dos vértices dos triângulos que compõem o quadrado 
        vertex_position = np.array([
            -0.5, -0.5, 0.0, 1.0, # Triângulo 1 (sentido anti-horário)
             0.5, -0.5, 0.0, 1.0,
            -0.5,  0.5, 0.0, 1.0,
             0.5, -0.5, 0.0, 1.0, # Triângulo 2 (sentido anti-horário)
             0.5,  0.5, 0.0, 1.0,
            -0.5,  0.5, 0.0, 1.0],
            dtype=np.float32)
        
        # posição dos vértices do quadrado 
        vertex_position_02 = np.array([
            -0.5, -0.5, 0.0, 1.0,  #vértice 0
             0.5, -0.5, 0.0, 1.0,  #vértice 1
             0.5,  0.5, 0.0, 1.0,  #vértice 2
            -0.5,  0.5, 0.0, 1.0], #vértice 3
            dtype=np.float32)
        
        # índices dos vértices dos triângulos que compõem o quadrado 
        vertex_indices = np.array([
             0, 1, 3,  # Triângulo 1 (sentido anti-horário)
             1, 2, 3], # Triângulo 2 (sentido anti-horário)
            dtype=np.uint32)
        
        # cria os objetos responsável por carregar os dados para a GPU
        # e renderizá-los como triângulos (primitiva padrão da classe)
        self.squareRenderer = ModelRenderer(vertex_position)
        self.squareIndicesRenderer = ModelRenderer(vertex_position_02,  vertex_indices=vertex_indices)
        
        # cria os objetos responsável por carregar os dados para a GPU e renderizá-los como linhas
        self.lineRenderer = ModelRenderer(vertex_position_02, primitive=ModelRenderer.LINES)
        self.lineLoopRenderer = ModelRenderer(vertex_position_02,  primitive=ModelRenderer.LINE_LOOP)
        
        # cria os objetos responsável por carregar os dados para a GPU e renderizá-los como pontos
        self.pointsRenderer = ModelRenderer(vertex_position_02,  primitive=ModelRenderer.POINTS)
        
        # cria um shader program simples
        self.shaderProgram = SimpleShaderProgram()
        
        # ativa o shader programa para configurar uma cor única para todos os vértices
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.release()

        # recupera o endereços da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        
        # configura os dados dos modelos para serem os dados de entrada do shader program
        self.squareRenderer.setVertexPositionLoc(position_loc)
        self.squareIndicesRenderer.setVertexPositionLoc(position_loc)
        self.lineRenderer.setVertexPositionLoc(position_loc)
        self.lineLoopRenderer.setVertexPositionLoc(position_loc)
        self.pointsRenderer.setVertexPositionLoc(position_loc)

    def paintGL(self):
        
        # configura a cor de background
        gl.glClearColor(0, 0, 0, 1)
        
        # limpa o background com a cor especificada
        gl.glClear(gl.GL_COLOR_BUFFER_BIT)
        
        # ativa o recurso de culling
        gl.glEnable(gl.GL_CULL_FACE)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # renderiza os objetos
        self.renderSquare()
        self.renderSquareWithIndices()
        self.renderLines()
        self.renderLineLoop()
        self.renderPoints()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
    
    def renderSquare(self):
        
        # atualiza o viewport para o objeto ser renderizado no canto superior esquerdo
        gl.glViewport(0, int(self.height / 2), int(self.width / 2), int(self.height / 2))
        
        # mudar a cor no shader e renderiza o quadrado
        self.shaderProgram.setUniformColor(np.array([0.0, 0.0, 0.5, 1.0], dtype=np.float32))
        self.squareRenderer.render()
        
        # mudar a cor no shader e renderiza o wireframe do quadrado
        self.shaderProgram.setUniformColor(np.array([1.0, 1.0, 1.0, 1.0], dtype=np.float32))
        self.squareRenderer.renderWireframe()
    
    def renderSquareWithIndices(self):
        
        # atualiza o viewport para o objeto ser renderizado no canto superior direito
        gl.glViewport(int(self.width / 2), int(self.height / 2), int(self.width / 2), int(self.height / 2))
        
        # mudar a cor no shader e renderiza o quadrado
        self.shaderProgram.setUniformColor(np.array([0.0, 0.5, 0.0, 1.0], dtype=np.float32))
        self.squareIndicesRenderer.render()
        
        # mudar a cor no shader e renderiza o wireframe do quadrado
        self.shaderProgram.setUniformColor(np.array([1.0, 1.0, 1.0, 1.0], dtype=np.float32))
        self.squareIndicesRenderer.renderWireframe()

    def renderLines(self):
        
        # atualiza o viewport para o objeto ser renderizado no lado esquerdo da metade inferior
        gl.glViewport(0, 0, int(self.width / 3), int(self.height / 2))
        
        # mudar a cor no shader e renderiza as linhas
        self.shaderProgram.setUniformColor(np.array([1.0, 0.0, 0.0, 1.0], dtype=np.float32))
        self.lineRenderer.render()
        
    def renderLineLoop(self):
        
        # atualiza o viewport para o objeto ser renderizado no centro da metade inferior
        gl.glViewport(int(self.width / 3), 0, int(self.width / 3), int(self.height / 2))
        
        # mudar a cor no shader e renderiza as linhas
        self.shaderProgram.setUniformColor(np.array([0.0, 1.0, 0.0, 1.0], dtype=np.float32))
        self.lineLoopRenderer.render()

    def renderPoints(self):
        
        # atualiza o viewport para o objeto ser renderizado no lado direito da metade inferior
        gl.glViewport(int(2 * self.width / 3), 0, int(self.width / 3), int(self.height / 2))
        
        # mudar a cor no shader e renderiza as linhas
        self.shaderProgram.setUniformColor(np.array([1.0, 1.0, 1.0, 1.0], dtype=np.float32))
        self.pointsRenderer.render()
        
    def resizeGL(self, width, height):
    
        # armazena a largura e a altura da janela
        self.width = width
        self.height = height

def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(640, 480)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
! jupyter nbconvert --to python 25_Formas_de_especificar_um_modelo.ipynb
%run -i 25_Formas_de_especificar_um_modelo.py

[NbConvertApp] Converting notebook 25_Formas_de_especificar_um_modelo.ipynb to python
[NbConvertApp] Writing 6532 bytes to 25_Formas_de_especificar_um_modelo.py


IndentationError: expected an indented block (25_Formas_de_especificar_um_modelo.py, line 120)